In [2]:
import warnings
warnings.filterwarnings('always')

import pandas as pd
import numpy as np

from sklearn.datasets import fetch_covtype
from sklearn import naive_bayes as nb
from sklearn import neural_network as nn

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split

import time

#from codes import cross_validate_classifier, \
#                  format_classifier_performance, \
#                  plot_confusion_matrix, metrics_wrapper

import codes as c

import importlib
importlib.reload(c)

<module 'codes' from '/Users/pmccauley/analysis/comp5318/comp5318_assignment_2/codes/__init__.py'>

In [ ]:
def class_count_split(X, y, count=250):
    '''
    Parameters:
        X = data
        y = labels
        
        count: number of each class to keep in training set, default 250, only 
                used if STYLE='equal'
                    
    Returns:
        X_train, y_train, X_test, y_test
    '''   
    
    #unique classes
    classes = np.unique(y)
    
    #empty array to store random pick indices 
    picks = np.empty([count, np.shape(classes)[0]])

    #loop through each class and randomly select <count> representatives
    for i, this_class in enumerate(classes):
        want = np.where(y == this_class)[0]
        picks[:,i] = np.random.choice(want,size=count, replace=False)

    #sort pick indices by the order they appear in the original table
    picks = np.sort(picks.flatten()).astype(int)
    
    X_test = X[picks,:]
    y_test = y[picks]
    
    X_train = np.delete(X,picks,axis=0)
    y_train = np.delete(y,picks)  
        
    return X_train, y_train, X_test, y_test

In [ ]:
def split_dataset(X, y, style='prop', count=250, test_size=0.1):
    '''
    Parameters:
        X = data
        y = labels
        
        style: 'prop' = classes proportionally represented in test dataset based
                        on proportions in training dataseg
                'equal' = equal number of class counts in training set
                'random' = random test/training partition
        
        count: number of each class to keep in training set, default 250, only 
                used if STYLE='equal'
        
        test_size: fraction of dataset used for test set, default 0.1, only used 
                    if STYLE='prop' or 'random'
                    
    Returns:
        X_train, y_train, X_test, y_test
    '''
    
    if style=='equal':
        X_train, y_train, X_test, y_test = class_count_split(X,y,count=count)
        
    if style=='prop':
        sss = StratifiedShuffleSplit(n_splits=1, test_size=test_size)
        for train_index, test_index in sss.split(X, y):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            
    if style=='random':
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
            
    return X_train, y_train, X_test, y_test

In [3]:
def read_covtype(crop=False, regularize=True):
    data = fetch_covtype()
    X = data['data']
    y = data['target']
    
    if crop==True:
        X = X[:,0:10]
    
    cnames = ['Spruce/Fir','Lodgepole Pine','Ponderosa Pine','Cottonwood/Willow','Aspen','Douglas-fir','Krummholz']
    cnums = np.arange(1,8)
    
    from sklearn.preprocessing import StandardScaler  
    scaler = StandardScaler()  
    # Don't cheat - fit only on training data
    scaler.fit(X)  
    X = scaler.transform(X)  
    # apply same transformation to test data
    #X_test = scaler.transform(X_test)  
    
    return X, y, cnames, cnums
    

In [4]:
X, y, cnames, cnums = read_covtype(crop=False)

In [9]:
X_train, y_train, X_test, y_test = c.split_dataset(X,y,regularize=True)
print(np.max(X_train), np.max(X))

511.32572789501137 440.0791595422305


In [ ]:
_classifier = nb.GaussianNB
confs = c.cross_validate_classifier(_classifier, X, y-1, kfold=1, style='equal')

In [ ]:
conf = np.mean(confs, axis=0)
print(conf.shape)
df_total, df_class, df_conf = c.metrics_wrapper(conf, cnames, do_display=True)
print(df_total['Overall Accuracy'][0])

In [ ]:
X_train, y_train, X_test, y_test = split_dataset(X,y, style='prop')
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

In [ ]:
_classifier = nb.GaussianNB()
_classifier = _classifier.fit(X_train, y_train)
y_pred = _classifier.predict(X_test)

In [ ]:
conf = c.construct_confusion_matrix(y_test-1, y_pred-1, dim=7)
df_total, df_class, df_conf = c.metrics_wrapper(conf, cnames, do_display=True)

In [10]:
#hidden layer number = mean([features,columns])
_classifier = nn.MLPClassifier(hidden_layer_sizes=[100])
_classifier = _classifier.fit(X_train, y_train)
y_pred = _classifier.predict(X_test)

In [ ]:

#layer_sizes = np.arange(2,202,2)
layer_sizes = np.arange(2,6,2)
runtimes = np.zeros(layer_sizes.shape[0])

print(layer_sizes,runtimes)

for i, layer_size in enumerate(layer_sizes):
    start = time.time()
    _classifier = nn.MLPClassifier(hidden_layer_sizes=layer_size)
    _classifier = _classifier.fit(X_train, y_train)
    y_pred = _classifier.predict(X_test)
    runtimes[i] = time.time()-start
    conf = c.construct_confusion_matrix(y_test-1, y_pred-1, dim=7)
    df_total, df_class, df_conf = c.metrics_wrapper(conf, cnames, do_display=False)
    df_tots = 
    

print(runtimes)

In [11]:
conf = c.construct_confusion_matrix(y_test-1, y_pred-1, dim=7)
df_total, df_class, df_conf = c.metrics_wrapper(conf, cnames, do_display=True)

Average/overall metrics:


,Average F-Meas,Average Precision,Average Recall,Overall Accuracy
Class-Averaged or Overall:,0.78,0.83,0.75,0.86


Class-specific metrics:


,Class,F-Meas,Precision,Recall
Confusion Matrix Index,,,,
0,Spruce/Fir,0.86,0.89,0.83
1,Lodgepole Pine,0.89,0.86,0.92
2,Ponderosa Pine,0.85,0.82,0.87
3,Cottonwood/Willow,0.74,0.88,0.65
4,Aspen,0.58,0.69,0.5
5,Douglas-fir,0.69,0.76,0.63
6,Krummholz,0.88,0.89,0.86


Confusion matrix (yellow = col max; red = row max):


Predicted,0,1,2,3,4,5,6
Actual,,,,,,,
0,17562,3423,2,0,14,6,177
1,1898,26021,112,1,143,122,34
2,1,209,3113,16,40,196,0
3,0,0,77,178,0,20,0
4,27,427,21,0,472,2,0
5,8,159,450,8,12,1100,0
6,258,21,0,0,0,0,1772


In [ ]:
blah = pd.concat([df_conf,df_conf], axis=1)
print(blah.shape, df_conf.shape)

In [ ]:
conf = c.construct_confusion_matrix(y_test-1, y_pred-1, dim=7)
df_total, df_class, df_conf = c.metrics_wrapper(conf, cnames, do_display=True)